In [1]:
import numpy as np
import math
from numpy import linalg as LA
from fractions import Fraction
import warnings
warnings.filterwarnings("ignore", category=np.VisibleDeprecationWarning) 

# 1. Euler2A

Euler2A[psi,theta,phi]-vraca matricu A = RZ(psi)RY(theta)RX(phi):

In [2]:
def Euler2A(phi, theta, psi):
    
    R_x = np.array([[1, 0, 0],
                    [0, math.cos(phi), -math.sin(phi)],
                    [0, math.sin(phi), math.cos(phi)]])

    R_y = np.array([[math.cos(theta), 0, math.sin(theta)],
                    [0, 1, 0],
                    [-math.sin(theta), 0, math.cos(theta)]])

    R_z = np.array([[math.cos(psi), -math.sin(psi), 0],
                    [math.sin(psi), math.cos(psi), 0],
                    [0, 0, 1]])


    A = (R_z.dot(R_y)).dot(R_x)
    return A

In [3]:
# A = Euler2A(-math.atan(1/4),-math.asin(8/9),math.atan(4))
# print("Matrica A za zadati test primer:")
# print(A)
# print("\n")

# #Broj indeksa je 105
# phi_test=2*math.pi/8*(105%7+1)
# theta_test=math.pi/17*(105%8+1)
# psi_test=2*math.pi/6*(105%5+1)
# TEST = Euler2A(phi_test,theta_test,psi_test)
# print("Matrica A za n koji je jednak mom broju indeksa 105:")
# print(TEST)


# 2. AxisAngle[A]

AxisAngle[A]-vraca jedinični vektor p=(px;py;pz)i ugao phi tako da A=Rp(phi):

In [4]:
def crossProduct(r, q):
    p = np.zeros((1, 3))
        
    p[0][0] = r[1]*q[2] - r[2]*q[1]
    p[0][1] = r[2]*q[0] - r[0]*q[2]
    p[0][2] = r[0]*q[1] - r[1]*q[0]

    return p

In [5]:
def AxisAngle(A):
    E=np.eye(3)
#     prvi_uslov = A.dot(A.T).round(5) == E
#     prvi_uslov = prvi_uslov.all()
#     drugi_uslov = np.linalg.det(A) == 1.0    
#     if not prvi_uslov or not drugi_uslov:
#         print("Matrica A nije matrica kretanja")
#         return -1, -1

    X = A - E

    r = np.array([X[0][0], X[0][1], X[0][2]])
    q = np.array([X[1][0], X[1][1], X[1][2]])

    p=crossProduct(r,q)
    #print(p)

    if p[0][0] == 0 and p[0][1] == 0 and p[0][2] == 0: 
        q = np.array([X[2][0], X[2][1], X[2][2]])

    p_norm = math.sqrt(p[0][0]**2 + p[0][1]**2 + p[0][2]**2)
    p = p * (1/p_norm)

    
    r_norm = math.sqrt(r[0]**2 + r[1]**2 + r[2]**2)
    u = r * (1/r_norm) 

    up = A.dot(u)

    cos_phi = np.dot(u, up)
    phi = math.acos(cos_phi)

    return p, phi

In [6]:
# p, phi1 = AxisAngle(A)
# print("Test primer")
# print("p:")
# print(p)
# print("\nUgao:")
# print(phi1)
# print("\n")

# print("Test primer za moj index")
# print("p:")
# p_test, phi1_test = AxisAngle(TEST)
# print(p_test)
# print("\nUgao:")
# print(phi1_test)


# 3. Rodrigez[p,phi]

Rodrigez[p,phi] - vraća matricu rotacije oko orjentisanog (jediničnog) vektora p za ugao phi .

In [7]:
def rodrigez(p, phi):

    E=np.eye(3)
#     uslov = int(p[0][0]**2+p[0][1]**2+p[0][2]**2)+1
#     if uslov!=1.0:
#         print("Vektor p nije jedinicni!")
#         return -1
    

    R = p.T.dot(p)
    Q = E - R
    R = R + math.cos(phi)*Q

    p_x = np.array([[0, -p[0][2], p[0][1]],
                    [p[0][2], 0, -p[0][0]],
                    [-p[0][1], p[0][0], 0]])

    R = R + math.sin(phi)*p_x

    return R


In [8]:
# print("Test primer")
# R = rodrigez(p,phi1)
# print(R)
# print("\n")
# print("Test primer za moj index")
# R_test = rodrigez(p_test,phi1_test)
# print(R_test)

# 4. A2Euler[A] 

A2Euler[A] - za datu ortogonalnu matricu A, det(A) = 1, vraca Ojlerove uglove psi,theta,phi redom.

In [9]:
def A2Euler(A):
#     uslov = np.linalg.det(A)
#     if uslov!=1.0:
#         print("Matrica A nije ortogonalna!")
#         return -1, -1, -1

    a = A[2][0]

    if a == 1:
        theta = -math.pi / 2
        phi = 0
        psi = math.atan2(-A[0][1], A[1][1])
        theta = math.pi / 2
        phi = 0
        psi = math.atan2(-A[0][1], A[1][1])
    else:
        theta = math.asin(-a)
        phi = math.atan2(A[2][1], A[2][2])
        psi = math.atan2(A[1][0], A[0][0])
    
    return phi, theta, psi

In [10]:
# print("Test primer:")
# phi, theta, psi = A2Euler(A)
# print("Phi: ", phi) 
# print("Theta: ", theta)
# print("Psi: ", psi)
# print("\n")


# print("Test primer za moj index")
# phi_test,theta_test,psi_test=A2Euler(TEST)
# print("Phi: ", phi_test) 
# print("Theta: ", theta_test)
# print("Psi: ", psi_test)

# 5. AxisAngle2Q[p,phi]

AxisAngle2Q[p,phi] - vraća jednični kvaternion q =(x,y,z,w) tako da Cq = Rp(phi).Vektor p je jednični.



In [11]:
def AxisAngle2Q(p, phi):
    uslov = (p[0][0]**2+p[0][1]**2+p[0][2]**2)
    if uslov==1.0:
        print("Vektor p nije jedinicni!")
        norma = math.sqrt(p[0][0]**2+p[0][1]**2+p[0][2]**2)
        p = p * (1/norma)

    w = math.cos(phi/2)

    r = math.sin(phi/2) * p

    q = np.array([r, w])
    
#[array([[ 0.23570226, -0.47140452,  0.47140452]]) 0.7071067811865476] da nam ne bi ispalo ovako radimo sledece:
    q = np.array([q[0][0][0], q[0][0][1], q[0][0][2], q[1]])

    return q    

In [12]:
# q = AxisAngle2Q(p, phi1)
# print("Kvaternion:")
# print(q)

# 6. Q2AxisAngle[q]

Q2AxisAngle[q]-vraća jedinični vektor p=(px,py,pz) i ugao phi tako da kvaternion predstavlja rotaciju Rp(phi),tj.
Cq=Rp(phi):

In [13]:
def Q2AxisAngle(q):
    uslov = (q[0]**2+q[1]**2+q[2]**2+q[3]**2).round(3)
    if uslov!=1.0:
        print("Kvaternion nije jedincni!")
        norma = math.sqrt(q[0]**2+q[1]**2+q[2]**2+q[3]**2)
        q = q * (1/norma)

    w = q[3]

    if w < 0:
        q = -q

    phi = 2*math.acos(w)

    if abs(w) == 1:
        p = np.array([1, 0, 0])
    else:
        v = np.array([q[0], q[1], q[2]])
        v_norma = math.sqrt(v[0]**2 + v[1]**2 + v[2]**2)
        p = v * (1/v_norma)

    return p, phi

In [14]:
# p, phi = Q2AxisAngle(q)
# print("Jedinicni vektor p i kvaternion:")
# print(p, phi) 

# TESTIRANJE                                                                      

1. Euler2A[psi,theta,phi]

In [15]:
A = Euler2A(-math.atan(1/4),-math.asin(8/9),math.atan(4))
print("Matrica A za zadati test primer:")
print(A)
print("\n")
#Broj indeksa je 105
phi_test=2*math.pi/8*(105%7+1)
theta_test=math.pi/17*(105%8+1)
psi_test=2*math.pi/6*(105%5+1)
TEST = Euler2A(phi_test,theta_test,psi_test)
print("Matrica A za n koji je jednak mom broju indeksa 105:")
print(TEST)

Matrica A za zadati test primer:
[[ 0.11111111 -0.88888889 -0.44444444]
 [ 0.44444444  0.44444444 -0.77777778]
 [ 0.88888889 -0.11111111  0.44444444]]


Matrica A za n koji je jednak mom broju indeksa 105:
[[ 0.46623611 -0.48465422  0.74009065]
 [ 0.80754464  0.57476783 -0.13233895]
 [-0.36124167  0.65935744  0.65935744]]


2. AxisAngle[A]


In [16]:
p, phi1 = AxisAngle(A)
print("Test primer")
print("p:")
print(p)
print("\nUgao:")
print(phi1)
print("\n")
print("Test primer za moj index")
print("p:")
p_test, phi1_test = AxisAngle(TEST)
print(p_test)
print("\nUgao:")
print(phi1_test)

Test primer
p:
[[ 0.33333333 -0.66666667  0.66666667]]

Ugao:
1.5707963267948966


Test primer za moj index
p:
[[0.42260674 0.58789008 0.68977445]]

Ugao:
1.213032325303286


3. Rodrigez[p,phi] 

In [17]:
print("Test primer")
R = rodrigez(p,phi1)
print(R)
print("\n")
print("Test primer za moj index")
R_test = rodrigez(p_test,phi1_test)
print(R_test)
#Kao sto je i trebalo dobili smo matricu 
#jednaku matrici A koju smo nasli pod 1)

Test primer
[[ 0.11111111 -0.88888889 -0.44444444]
 [ 0.44444444  0.44444444 -0.77777778]
 [ 0.88888889 -0.11111111  0.44444444]]


Test primer za moj index
[[ 0.46623611 -0.48465422  0.74009065]
 [ 0.80754464  0.57476783 -0.13233895]
 [-0.36124167  0.65935744  0.65935744]]


4. A2Euler[A]

In [18]:
print("Test primer:")
phi, theta, psi = A2Euler(A)
print("Phi: ", phi) 
print("Theta: ", theta)
print("Psi: ", psi)
print("\n")
print("Test primer za moj index")
phi_test,theta_test,psi_test=A2Euler(TEST)
print("Phi: ", phi_test) 
print("Theta: ", theta_test)
print("Psi: ", psi_test)

Test primer:
Phi:  -0.24497866312686414
Theta:  -1.09491407713448
Psi:  1.3258176636680326


Test primer za moj index
Phi:  0.7853981633974483
Theta:  0.36959913571644626
Psi:  1.0471975511965976


5. AxisAngle2Q[p,phi]

In [19]:
q = AxisAngle2Q(p, phi1)
print("Kvaternion:")
print(q)
print("Test primer za moj index")
qq=AxisAngle2Q(p_test,phi1_test)
print(qq)

Kvaternion:
[ 0.23570226 -0.47140452  0.47140452  0.70710678]
Test primer za moj index
[0.24088942 0.33510233 0.39317728 0.82163882]


6. Q2AxisAngle[q] 

In [22]:
p, phi = Q2AxisAngle(q)
print("Jedinicni vektor p i kvaternion:")
print(p, phi)
print("Test primer za moj index")
pp,pphi=Q2AxisAngle(qq)
print(pp,pphi)
#Dobili smo isto kao i u AxisAngle[A].

Jedinicni vektor p i kvaternion:
[ 0.33333333 -0.66666667  0.66666667] 1.5707963267948966
Test primer za moj index
[0.42260674 0.58789008 0.68977445] 1.2130323253032858
